### Este notebook tem a função de formar um dataframe com todas as informações dos setores censitários sobre população, domicílios e rendas domiciliares do Brasil. Este arquivo será exportado depois lido em outro script para a formação de um dashboard para visualização dos dados.

---


#### Importação de bibliotecas


In [1]:
import os  # Criação de estrutura de pastas e manuseamento de arquivos
from ftplib import FTP  # Biblioteca para download dos arquivos de servidor ftp
from zipfile import ZipFile  # Biblioteca para descompactar os arquivos
import pandas as pd  # Tratamento dos dados
import geopandas as gpd  # Tratamento dos dados geográficos

---


#### Estrutura de pastas


In [2]:
parent_dir = os.getcwd()  # Identificação do diretório de trabalho

"""
Criação dos diretórios para arquivamento dos arquivos
data: Arquivos baixados e a serem utilizados para tratamento.
artifacts: Todos os arquivos criados para utilização e visualização.
"""

directories = ["data", "artifacts"]
for directory in directories:
    path = os.path.join(parent_dir, directory)
    try:
        os.mkdir(path)
        print(f"Directory '{directory}' created.")
    except FileExistsError:
        print(f"Directory '{directory}' already created!")

data_dir = os.path.join(parent_dir, directories[0])
artifacts_dir = os.path.join(parent_dir, directories[1])

Directory 'data' already created!
Directory 'artifacts' already created!


#### Download de arquivos


Acessando os servidores FTP e baixando os arquivos.


In [3]:
ftp_server = "ftp.ibge.gov.br"  # Servidor FTP do IBGE
ftp_dir = "Censos/Censo_Demografico_2010/Resultados_do_Universo/"\
    "Agregados_por_Setores_Censitarios/"  # Diretório a ser acessado
# O servidor aceita acesso público, portanto user e password desnecessários
user = "anonymous"  # Inserir anonymous para entrar no servidor
passwd = ""

# Configuração da variável de acesso ao servidor
ftp = FTP(ftp_server, user, passwd)
ftp.cwd(ftp_dir)  # Acessando o diretório escolhido
files = ftp.nlst()  # variável contendo o conteúdo do diretório

ftp.dir()  # Listagem do conteúdo do diretório

-rwxrwxr-x    1 ftp      ftp          2186 Dec 12 09:29 1_Atualizacoes_20241211.txt
-rwxrwxr-x    1 ftp      ftp       5579523 Oct 30  2023 AC_20231030.zip
-rwxrwxr-x    1 ftp      ftp      22399914 Oct 30  2023 AL_20231030.zip
-rwxrwxr-x    1 ftp      ftp      31064700 Oct 30  2023 AM_20231030.zip
-rwxrwxr-x    1 ftp      ftp       5194028 Oct 30  2023 AP_20231030.zip
-rwxrwxr-x    1 ftp      ftp      126886126 Oct 30  2023 BA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      71781739 Oct 30  2023 CE_20231030.zip
-rwxrwxr-x    1 ftp      ftp      21585996 Oct 30  2023 DF_20231030.zip
-rwxrwxr-x    1 ftp      ftp      34133643 Oct 30  2023 Documentacao_Agregado_dos_Setores_2010_20231030.zip
-rwxrwxr-x    1 ftp      ftp      37084676 Oct 30  2023 ES_20231030.zip
-rwxrwxr-x    1 ftp      ftp      53664403 Oct 30  2023 GO_20231030.zip
-rwxrwxr-x    1 ftp      ftp      48421505 Oct 30  2023 MA_20231030.zip
-rwxrwxr-x    1 ftp      ftp      169948641 Oct 30  2023 MG_20231030.zip
-rwxrwxr-x    

In [4]:
files_check = [filename for filename in os.listdir(data_dir)]  # lista dos arquivos no diretório

In [5]:
# Loop para download dos arquivos no servidor FTP
for file in files:
    if file not in files_check:
        with open(os.path.join(data_dir, file), 'wb') as fp:
            ftp.retrbinary('RETR ' + file, fp.write)
            print(file+" downloaded to "+data_dir)

ftp.quit()  # fechamento da conexão com o servidor FTP

'221 Goodbye.'

In [6]:
# Acessando o servidor ftp com as informações geográficas dos setores censitários
ftp_server = "geoftp.ibge.gov.br"
ftp_dir = "organizacao_do_territorio/malhas_territoriais/"\
    "malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/"\
    "setores_censitarios_shp/"
user = "anonymous"
passwd = ""

ftp = FTP(ftp_server, user, passwd)
ftp.cwd(ftp_dir)
ftp.dir()

drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 1_leia_me
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ac
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 al
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 am
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ap
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ba
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ce
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 df
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 es
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 go
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ma
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mg
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 ms
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 mt
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pa
drwxrwxr-x    2 ftp      ftp          4096 Jun 14  2016 pb
drwxrwxr-x    2 ftp      ftp          4096 Jun 14

In [7]:
UFs_geo = ftp.nlst() # Listagem dos diretórios a serem verificadas

# variáveis com aa extensões dos arquivos a serem baixados
extension_geo = "_setores_censitarios.zip"
extension_geo_1 = "_setores _censitarios.zip"

files_check = [filename for filename in os.listdir(data_dir)]

In [8]:
# Loop entre todos os diretórios dos estados para download dos arquivos geográficos
for UF_geo in UFs_geo:
    ftp.cwd(UF_geo)
    files = ftp.nlst()
    for file in files:
        if ((file == UF_geo+extension_geo) or (file == UF_geo+extension_geo_1))\
            and (file not in files_check) :
            with open(os.path.join(data_dir, file), 'wb') as fp:
                ftp.retrbinary('RETR ' + file, fp.write)
                print(file+" downloaded to "+data_dir)
    ftp.cwd("..")  # Voltando ao diretório anterior para repetição do processo
ftp.quit()

'221 Goodbye.'

---


#### Extração dos arquivos compactados.


In [9]:
data_dir_files = os.listdir(data_dir)  # Lista dos arquivos no diretório 'data'
data_dir_files

['1_Atualizacoes_20241211.txt',
 'AC_20231030',
 'AC_20231030.zip',
 'ac_setores_censitarios',
 'ac_setores_censitarios.zip',
 'AL_20231030',
 'AL_20231030.zip',
 'al_setores_censitarios',
 'al_setores_censitarios.zip',
 'AM_20231030',
 'AM_20231030.zip',
 'am_setores_censitarios',
 'am_setores_censitarios.zip',
 'AP_20231030',
 'AP_20231030.zip',
 'ap_setores_censitarios',
 'ap_setores_censitarios.zip',
 'BA_20231030',
 'BA_20231030.zip',
 'ba_setores_censitarios',
 'ba_setores_censitarios.zip',
 'CE_20231030',
 'CE_20231030.zip',
 'ce_setores_censitarios',
 'ce_setores_censitarios.zip',
 'DF_20231030',
 'DF_20231030.zip',
 'df_setores_censitarios',
 'df_setores_censitarios.zip',
 'Documentacao_Agregado_dos_Setores_2010_20231030',
 'Documentacao_Agregado_dos_Setores_2010_20231030.zip',
 'ES_20231030',
 'ES_20231030.zip',
 'es_setores_censitarios',
 'es_setores_censitarios.zip',
 'GO_20231030',
 'GO_20231030.zip',
 'go_setores _censitarios',
 'go_setores _censitarios.zip',
 'MA_2023103

In [10]:
# Loop para extração dos arquivos compactados
for file in data_dir_files:
    try:
        if file.split(".")[1] == "zip":  # Verificação se são arquivos .zip
            file_zipped = ZipFile(os.path.join(data_dir, file))

            new_folder_path = os.path.join(data_dir, file.split(
                ".zip")[0])  # Diretório para receber os arquivos

            # Criação do diretório caso não exista
            if not os.path.exists(new_folder_path):
                file_zipped.extractall(new_folder_path)

            file_zipped.close()
    except:
        print(f"{file} - Not a zipped file")

AC_20231030 - Not a zipped file
ac_setores_censitarios - Not a zipped file
AL_20231030 - Not a zipped file
al_setores_censitarios - Not a zipped file
AM_20231030 - Not a zipped file
am_setores_censitarios - Not a zipped file
AP_20231030 - Not a zipped file
ap_setores_censitarios - Not a zipped file
BA_20231030 - Not a zipped file
ba_setores_censitarios - Not a zipped file
CE_20231030 - Not a zipped file
ce_setores_censitarios - Not a zipped file
DF_20231030 - Not a zipped file
df_setores_censitarios - Not a zipped file
Documentacao_Agregado_dos_Setores_2010_20231030 - Not a zipped file
ES_20231030 - Not a zipped file
es_setores_censitarios - Not a zipped file
GO_20231030 - Not a zipped file
go_setores _censitarios - Not a zipped file
MA_20231030 - Not a zipped file
ma_setores_censitarios - Not a zipped file
MG_20231030 - Not a zipped file
mg_setores_censitarios - Not a zipped file
MS_20231030 - Not a zipped file
ms_setores_censitarios - Not a zipped file
MT_20231030 - Not a zipped file

Na extração são dados arquivos tipo .csv e .xls. Neste caso, será mantido apenas os arquivos tipo .csv

In [11]:
for dirpath, dirnames, filenames in os.walk(data_dir):
    for filename in filenames:
        if (filename.endswith(".xls")) or (filename.endswith(".XLS"))\
            or (filename.endswith(".xlsx")):  # Checagem dos arquivos tipo excel
                file_path = os.path.join(dirpath, filename)  # Obtendo o caminho do arquivo
                os.remove(file_path)  # Deletando o arquivo
                print(f"Deleted: {file_path}")

In [12]:
# Formação da lista de arquivos contido no diretório data
files_list = []

for dirpath, dirnames, filenames in os.walk(data_dir):
    for filename in filenames:
        file_path = os.path.join(dirpath, filename)
        files_list.append(file_path)

files_list

['c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\1_Atualizacoes_20241211.txt',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AC_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\ac_setores_censitarios.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AL_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\al_setores_censitarios.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AM_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\am_setores_censitarios.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AP_20231030.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\ap_setores_censitarios.zip',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\BA

---


#### Tratamento de dados


In [13]:
# Passando a lista de arquivos para um dataframe para utilização mais fácil dos arquivos
pd.set_option("max_colwidth", 150)
files_list_df = pd.DataFrame(files_list, columns=['filepaths'])
files_list_df

,filepaths
0,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\1_Atualizacoes_20241211.txt
1,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\AC_20231030.zip
2,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ac_setores_censitarios.zip
3,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\AL_20231030.zip
4,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\al_setores_censitarios.zip
...,...
888,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\TO_20231030\Base informaçoes setores2010 universo TO\CSV\ResponsavelRenda_TO.csv
889,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\to_setores_censitarios\17SEE250GC_SIR.dbf
890,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\to_setores_censitarios\17SEE250GC_SIR.prj
891,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\to_setores_censitarios\17SEE250GC_SIR.shp


Agora, será necessário utilizar os arquivos necessários do censo para retirar a população e renda domiciliar de cada setor censitário, para isso serão utilizados três arquivos de acordo com a documentação e eles possuem os seguintes formatos Pessoa13_UF.csv, Domicilio01_UF e DomicilioRenda_UF.

In [14]:
mask_domiciliorenda = files_list_df['filepaths'].str.contains('domiciliorenda', case=False, na=False)
mask_domicilio = files_list_df['filepaths'].str.contains('domicilio01', case=False, na=False)
mask_pop = files_list_df['filepaths'].str.contains('basico', case=False, na=False)

In [15]:
# Obtendo todos os caminhos para os arquivos necessários
filepaths = files_list_df[mask_domicilio | mask_domiciliorenda | mask_pop]
filepaths = filepaths['filepaths'].to_list()

In [16]:
filepaths

['c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AC_20231030\\Base informaçoes setores2010 universo AC\\CSV\\Basico_AC.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AC_20231030\\Base informaçoes setores2010 universo AC\\CSV\\Domicilio01_AC.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AC_20231030\\Base informaçoes setores2010 universo AC\\CSV\\DomicilioRenda_AC.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AL_20231030\\Base informaçoes setores2010 universo AL\\CSV\\Basico_AL.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AL_20231030\\Base informaçoes setores2010 universo AL\\CSV\\Domicilio01_AL.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\\setores_censitarios\\data\\AL_20231030\\Base informaçoes setores2010 universo AL\\CSV\\DomicilioRenda_AL.csv',
 'c:\\Users\\guicr\\OneDrive\\Carreira\\Portfolio\

In [17]:
# Função para definir o separador correto para cada arquivo
def create_df(file):
    df = pd.read_csv(file, encoding='iso-8859-1', sep=';')
    if df.shape[1] < 2:
        df = pd.read_csv(file, encoding='iso-8859-1', sep=',')
    return df

# Função para extrair texto de comparação de arquivo
def filename_finder(text):
    start = text.rfind('\\')+1
    end = text.find('.')
    return text[start:end]

In [18]:
# Loop entre cada tipo de arquivo relativo a população, renda domiciliar e domicílios para formação de um único dataframe
df_domicilio = pd.DataFrame(columns=['Cod_setor', 'V001'])
df_domicilio_renda = pd.DataFrame(columns=['Cod_setor', 'V002'])
df_populacao = pd.DataFrame(columns=['Cod_setor','Nome_da_UF ','Nome_do_municipio',
                                     'Nome_do_distrito','Nome_do_subdistrito',
                                     'Nome_do_bairro','V002'])
for file in filepaths:
    if 'domicilio01' in filename_finder(file).lower():
        df_domicilio_temp = create_df(file)
        df_domicilio_temp = df_domicilio_temp[['Cod_setor', 'V001']]
        df_domicilio = pd.concat([df_domicilio, df_domicilio_temp])
    elif 'domiciliorenda' in filename_finder(file).lower():
        df_dom_renda_temp = create_df(file)
        df_dom_renda_temp = df_dom_renda_temp[['Cod_setor', 'V002']]
        df_domicilio_renda = pd.concat([df_domicilio_renda, df_dom_renda_temp])
    elif 'basico' in filename_finder(file).lower():
        df_populacao_temp = create_df(file)
        df_populacao_temp = df_populacao_temp[['Cod_setor','Nome_da_UF ','Nome_do_municipio',
                                     'Nome_do_distrito','Nome_do_subdistrito',
                                     'Nome_do_bairro','V002']]
        df_populacao = pd.concat([df_populacao, df_populacao_temp])

C:\Users\guicr\AppData\Local\Temp\ipykernel_13636\14300837.py:3: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set l

In [19]:
df_domicilio.rename(columns={'V001': 'domicilios'}, inplace=True)
df_domicilio_renda.rename(columns={'V002': 'renda'}, inplace=True)
df_populacao.rename(columns={'V002': 'populacao'}, inplace=True)

In [20]:
# Mesclando os dataframes
df_drp = pd.merge(df_domicilio, df_domicilio_renda, on='Cod_setor')
df_drp = pd.merge(df_drp, df_populacao, on='Cod_setor')

In [21]:
# Transformando os valores em numéricos, evitando a coluna 'Cod_setor'
for column in df_drp.columns:
    if column in ['populacao', 'domicilios', 'renda']:
        df_drp[column] = pd.to_numeric(df_drp[column], errors='coerce') # Colocando tipo null em não numérico

In [22]:
df_drp.isnull().sum()

Cod_setor                 0
domicilios                0
renda                  6302
Nome_da_UF                0
Nome_do_municipio         0
Nome_do_distrito          0
Nome_do_subdistrito       0
Nome_do_bairro            0
populacao               773
dtype: int64

In [23]:
# df_drp[df_drp.isnull().any(axis=1)]
df_drp[df_drp['renda'].isnull()]

,Cod_setor,domicilios,renda,Nome_da_UF,Nome_do_municipio,Nome_do_distrito,Nome_do_subdistrito,Nome_do_bairro,populacao
20,120005405000007,23,NaN,Acre,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL (todos os setores),152.0
21,120005405000008,1,NaN,Acre,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL (todos os setores),7.0
52,120010405000027,11,NaN,Acre,BRASILÉIA,BRASILÉIA,BRASILÉIA,BRASILÉIA (todos os setores),61.0
55,120010405000030,2,NaN,Acre,BRASILÉIA,BRASILÉIA,BRASILÉIA,BRASILÉIA (todos os setores),13.0
60,120013805000004,25,NaN,Acre,BUJARI,BUJARI,BUJARI,BUJARI (todos os setores),84.0
...,...,...,...,...,...,...,...,...,...
310027,172110905000027,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),20.0
310030,172110905000031,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),25.0
310037,172110905000038,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),17.0
310038,172110905000039,2,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),17.0


Verifica-se que alguns dados da coluna renda estão omitidos. Isto acontece, pois, segundo a documentação do IBGE, os participantes não desejam a divulgação dos dados. Os valores serão alterados para zero.

In [24]:
df_drp[df_drp.isnull().any(axis=1)]

,Cod_setor,domicilios,renda,Nome_da_UF,Nome_do_municipio,Nome_do_distrito,Nome_do_subdistrito,Nome_do_bairro,populacao
20,120005405000007,23,NaN,Acre,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL (todos os setores),152.0
21,120005405000008,1,NaN,Acre,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL,ASSIS BRASIL (todos os setores),7.0
52,120010405000027,11,NaN,Acre,BRASILÉIA,BRASILÉIA,BRASILÉIA,BRASILÉIA (todos os setores),61.0
55,120010405000030,2,NaN,Acre,BRASILÉIA,BRASILÉIA,BRASILÉIA,BRASILÉIA (todos os setores),13.0
60,120013805000004,25,NaN,Acre,BUJARI,BUJARI,BUJARI,BUJARI (todos os setores),84.0
...,...,...,...,...,...,...,...,...,...
310027,172110905000027,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),20.0
310030,172110905000031,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),25.0
310037,172110905000038,4,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),17.0
310038,172110905000039,2,NaN,Tocantins,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA,TOCANTÍNIA (todos os setores),17.0


In [25]:
df_drp.fillna(0, inplace=True)

In [26]:
df_drp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310120 entries, 0 to 310119
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Cod_setor            310120 non-null  object 
 1   domicilios           310120 non-null  int64  
 2   renda                310120 non-null  float64
 3   Nome_da_UF           310120 non-null  object 
 4   Nome_do_municipio    310120 non-null  object 
 5   Nome_do_distrito     310120 non-null  object 
 6   Nome_do_subdistrito  310120 non-null  object 
 7   Nome_do_bairro       310120 non-null  object 
 8   populacao            310120 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 21.3+ MB


In [27]:
df_drp.reset_index(drop=True, inplace=True)

In [28]:
df_drp.sample(8)

,Cod_setor,domicilios,renda,Nome_da_UF,Nome_do_municipio,Nome_do_distrito,Nome_do_subdistrito,Nome_do_bairro,populacao
274628,351870105000131,301,916365.0,São Paulo,GUARUJÁ,GUARUJÁ,GUARUJÁ,Santo Antônio 2,1012.0
102976,315630405000005,245,580844.0,Minas Gerais,RODEIRO,RODEIRO,RODEIRO,RODEIRO (todos os setores),883.0
305242,355280905000241,212,366137.0,São Paulo,TABOÃO DA SERRA,TABOÃO DA SERRA,TABOÃO DA SERRA,TABOÃO DA SERRA (todos os setores),740.0
297530,354800505000001,280,597379.0,São Paulo,SANTO ANTÔNIO DE POSSE,SANTO ANTÔNIO DE POSSE,SANTO ANTÔNIO DE POSSE,SANTO ANTÔNIO DE POSSE (todos os setores),775.0
55351,320313005000019,193,291175.0,Espírito Santo,JOÃO NEIVA,JOÃO NEIVA,JOÃO NEIVA,JOÃO NEIVA (todos os setores),627.0
127459,150720105000007,180,104054.0,Pará,SÃO DOMINGOS DO CAPIM,SÃO DOMINGOS DO CAPIM,SÃO DOMINGOS DO CAPIM,SÃO DOMINGOS DO CAPIM (todos os setores),846.0
183510,330455705130067,132,1128491.0,Rio de Janeiro,RIO DE JANEIRO,RIO DE JANEIRO,TIJUCA,Tijuca,364.0
158000,410690205070260,184,462673.0,Paraná,CURITIBA,CURITIBA,ADMINISTRAÇÃO REGIONAL PINHEIRINHO (R.8.PN),Cidade Industrial de Curitiba,613.0


---


#### Tratamento dos dados geográficos


Aqui, forma-se o dataframe das informações geográficas para ter a localização dos setores e cálculo da densidade demográfica.

In [29]:
mask_shape = files_list_df['filepaths'].str.contains('.shp', case=False, na=False)
shp_files = files_list_df[mask_shape]

In [30]:
shp_files

,filepaths
85,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ac_setores_censitarios\12SEE250GC_SIR.shp
115,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\al_setores_censitarios\27SEE250GC_SIR.shp
145,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\am_setores_censitarios\13SEE250GC_SIR.shp
175,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ap_setores_censitarios\16SEE250GC_SIR.shp
205,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ba_setores_censitarios\29SEE250GC_SIR.shp
235,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ce_setores_censitarios\23SEE250GC_SIR.shp
265,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\df_setores_censitarios\53SEE250GC_SIR.shp
295,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\es_setores_censitarios\32SEE250GC_SIR.shp
325,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\go_setores _censitarios\52SEE250GC_SIR.shp
355,c:\Users\guicr\OneDrive\Carreira\Portfolio\setores_censitarios\data\ma_setores_censitarios\21SEE250GC_SIR.shp


In [31]:
# Função para obtenção da área
def get_area(df, chunk_size=100):
    df['area_ha'] = 0.0
    for start in range(0, len(df), chunk_size):
        end = min(start + chunk_size, len(df)) 
        epsg = df.loc[start:end, 'geometry'].set_crs(4674).estimate_utm_crs('SIRGAS 2000').to_epsg()
        utm_geometry = df.to_crs(epsg=epsg)
        df['area_ha'] = utm_geometry.area / 1e4
    return df
        

for i, file in enumerate(shp_files.iloc[:,0].values):
    if i == 0:
        br_gdf = gpd.read_file(file)
        br_gdf = get_area(br_gdf)
    else:
        temp_gdf = gpd.read_file(file)
        temp_gdf = get_area(temp_gdf)
        br_gdf= pd.concat([br_gdf, temp_gdf])

In [32]:
br_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 316574 entries, 0 to 2114
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ID          316574 non-null  float64 
 1   CD_GEOCODI  316574 non-null  object  
 2   TIPO        316574 non-null  object  
 3   CD_GEOCODB  106538 non-null  object  
 4   NM_BAIRRO   106538 non-null  object  
 5   CD_GEOCODS  316574 non-null  object  
 6   NM_SUBDIST  53318 non-null   object  
 7   CD_GEOCODD  316574 non-null  object  
 8   NM_DISTRIT  316574 non-null  object  
 9   CD_GEOCODM  316574 non-null  object  
 10  NM_MUNICIP  316574 non-null  object  
 11  NM_MICRO    316574 non-null  object  
 12  NM_MESO     316574 non-null  object  
 13  geometry    316574 non-null  geometry
 14  area_ha     316574 non-null  float64 
 15  ID1         68296 non-null   float64 
dtypes: float64(3), geometry(1), object(12)
memory usage: 41.1+ MB


In [62]:
br_gdf.rename(columns={'CD_GEOCODI':'Cod_setor'}, inplace=True)
br_gdf.columns

Index(['ID', 'Cod_setor', 'TIPO', 'CD_GEOCODB', 'NM_BAIRRO', 'CD_GEOCODS',
       'NM_SUBDIST', 'CD_GEOCODD', 'NM_DISTRIT', 'CD_GEOCODM', 'NM_MUNICIP',
       'NM_MICRO', 'NM_MESO', 'geometry', 'area_ha', 'ID1'],
      dtype='object')

In [63]:
br_gdf.reset_index(drop=True, inplace=True)

In [53]:
br_gdf.to_parquet('./artifacts/br_all_gdf.parquet')

In [64]:
br_gdf.sample(8).T

,59994,168478,253562,277460,140335,299528,247083,74906
ID,65223.0,17966.0,72289.0,96236.0,26431.0,118859.0,65832.0,4996.0
Cod_setor,320530905090067,411990525000003,350925405000035,353800605000002,260620010000006,355030847000211,350070905000022,210830605000018
TIPO,URBANO,RURAL,RURAL,URBANO,URBANO,URBANO,URBANO,RURAL
CD_GEOCODB,320530905038,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM_BAIRRO,Itararé,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CD_GEOCODS,32053090509,41199052500,35092540500,35380060500,26062001000,35503084700,35007090500,21083060500
NM_SUBDIST,REGIÃO ADMINISTRATIVA IV - MARUÍPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CD_GEOCODD,320530905,411990525,350925405,353800605,260620010,355030847,350070905,210830605
NM_DISTRIT,VITÓRIA,UVAIA,CAJATI,PINDAMONHANGABA,PONTA DE PEDRAS,JOSÉ BONIFÁCIO,AGUDOS,PENALVA
CD_GEOCODM,3205309,4119905,3509254,3538006,2606200,3550308,3500709,2108306


In [65]:
br_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 316574 entries, 0 to 316573
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ID          316574 non-null  float64 
 1   Cod_setor   316574 non-null  object  
 2   TIPO        316574 non-null  object  
 3   CD_GEOCODB  106538 non-null  object  
 4   NM_BAIRRO   106538 non-null  object  
 5   CD_GEOCODS  316574 non-null  object  
 6   NM_SUBDIST  53318 non-null   object  
 7   CD_GEOCODD  316574 non-null  object  
 8   NM_DISTRIT  316574 non-null  object  
 9   CD_GEOCODM  316574 non-null  object  
 10  NM_MUNICIP  316574 non-null  object  
 11  NM_MICRO    316574 non-null  object  
 12  NM_MESO     316574 non-null  object  
 13  geometry    316574 non-null  geometry
 14  area_ha     316574 non-null  float64 
 15  ID1         68296 non-null   float64 
dtypes: float64(3), geometry(1), object(12)
memory usage: 38.6+ MB


Extraindo as colunas de interesse do geodataframe.

In [66]:
stg_br_gdf = br_gdf.loc[:,['Cod_setor', 'geometry', 'area_ha']]

In [75]:
stg_br_gdf['Cod_setor'] = stg_br_gdf['Cod_setor'].astype('str')

In [76]:
stg_br_gdf.sample(10)

,Cod_setor,geometry,area_ha
78231,211230810000012,"POLYGON ((-44.73865 -5.72719, -44.73153 -5.72139, -44.73140 -5.72128, -44.73121 -5.72112, -44.72719 -5.71720, -44.72715 -5.71716, -44.72706 -5.717...",117.223045
218277,431306005000002,"POLYGON ((-50.90932 -29.58121, -50.90932 -29.58121, -50.90787 -29.58142, -50.90682 -29.58156, -50.90682 -29.58156, -50.90708 -29.58301, -50.90660 ...",26.350785
124279,150140260000035,"POLYGON ((-48.46910 -1.30690, -48.46906 -1.30718, -48.46902 -1.30776, -48.46901 -1.30791, -48.46889 -1.30897, -48.46885 -1.30941, -48.46919 -1.309...",7.040305
211432,430460605000207,"POLYGON ((-51.21697 -29.92228, -51.21534 -29.92310, -51.21162 -29.92505, -51.20997 -29.92791, -51.20888 -29.92875, -51.20720 -29.93041, -51.20663 ...",115.820304
311198,355250210000060,"POLYGON ((-46.29346 -23.50840, -46.29291 -23.50865, -46.29238 -23.50886, -46.29377 -23.51091, -46.29399 -23.51082, -46.29439 -23.51066, -46.29462 ...",3.479144
19676,291350705000008,"POLYGON ((-40.09020 -14.74957, -40.08860 -14.75193, -40.08843 -14.75186, -40.08783 -14.75165, -40.08782 -14.75170, -40.08788 -14.75214, -40.08798 ...",35.198437
75499,210970005000001,"POLYGON ((-45.34796 -7.13203, -45.34487 -7.13171, -45.34035 -7.13376, -45.34027 -7.13397, -45.34079 -7.13745, -45.33776 -7.14036, -45.33957 -7.141...",103.992964
312378,355410205000149,"POLYGON ((-45.56393 -23.05436, -45.56528 -23.05234, -45.56482 -23.05202, -45.56431 -23.05173, -45.56385 -23.05146, -45.56336 -23.05118, -45.56285 ...",6.649281
149877,220240605000011,"POLYGON ((-41.96053 -4.44044, -41.95744 -4.44426, -41.95877 -4.46718, -41.95109 -4.47575, -41.95151 -4.47663, -41.95217 -4.47798, -41.95257 -4.478...",4970.094104
43440,230780905000022,"POLYGON ((-40.14945 -3.10836, -40.14947 -3.10864, -40.14943 -3.10896, -40.14995 -3.10928, -40.15013 -3.10994, -40.15009 -3.11050, -40.14993 -3.110...",26.299117


---

Ao analisar as informações percebe-se que todos os dados necessários estão completos sem valores faltantes.


O cálculo de área é realizado na projeção UTM, portanto será necessário realizar a mudança de coordenada geográfica para cartesiana.


O retorno para a coordenada geográfica é necessária para a representação visual em mapas posteriormente.


    Consolidação dos dados geográficos com os dados dos domicílios.


A dimensionalidade dos dados geoográficos, também, é maior do que os dados dos domicílios. Portanto, mesclam-se os dataframes a atualizar somente para os dados de menor dimensionalidade.


In [83]:
df_drp['Cod_setor'] = df_drp['Cod_setor'].astype('str')

In [85]:
br = pd.merge(
    df_drp, stg_br_gdf, how='inner', on='Cod_setor')

In [86]:
br.head().T

,0,1,2,3,4
Cod_setor,120001305000001,120001305000002,120001305000003,120001305000004,120001305000005
domicilios,282,344,483,53,96
renda,601805.0,385433.0,532208.0,58853.0,112979.0
Nome_da_UF,Acre,Acre,Acre,Acre,Acre
Nome_do_municipio,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA
Nome_do_distrito,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA
Nome_do_subdistrito,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA,ACRELÂNDIA
Nome_do_bairro,ACRELÂNDIA (todos os setores),ACRELÂNDIA (todos os setores),ACRELÂNDIA (todos os setores),ACRELÂNDIA (todos os setores),ACRELÂNDIA (todos os setores)
populacao,957.0,1203.0,1700.0,182.0,305.0
geometry,"POLYGON ((-67.0493020893104 -10.070079922031299, -67.0499304027322 -10.074666610010398, -67.0503196154562 -10.077507862895699, -67.0518683041132 -...","POLYGON ((-67.0542077590318 -10.0911169390186, -67.054785367352 -10.089942017435199, -67.0551067155382 -10.089239812139398, -67.0550591083995 -10....","POLYGON ((-67.0764013172763 -10.0882102164659, -67.07640102518201 -10.0881675776819, -67.0764010515928 -10.088162506749, -67.07640122782729 -10.08...","POLYGON ((-67.0363059740346 -10.0056624233594, -67.03780641520531 -10.0072208949661, -67.03817115985309 -10.0078674877507, -67.0382470727329 -10.0...","POLYGON ((-67.053076677153 -9.94381954451563, -67.0475772671881 -9.9415561246586, -67.0444774642514 -9.9404329705857, -67.0417598640462 -9.9395780..."


In [87]:
br.sample(8).T

,58083,192679,52247,221497,295303,125842,178433,263860
Cod_setor,320520015090031,330490405000432,530010805250149,431720205000098,354520905000105,150548605000019,330340105000039,350570805000115
domicilios,157,307,140,70,276,111,306,74
renda,364843.0,385894.0,213583.0,140089.0,589470.0,72610.0,483067.0,0.0
Nome_da_UF,Espírito Santo,Rio de Janeiro,Distrito Federal,Rio Grande do Sul,São Paulo,Pará,Rio de Janeiro,São Paulo
Nome_do_municipio,VILA VELHA,SÃO GONÇALO,BRASÍLIA,SANTA ROSA,SALTO,PACAJÁ,NOVA FRIBURGO,BARUERI
Nome_do_distrito,IBES,SÃO GONÇALO,BRASÍLIA,SANTA ROSA,SALTO,PACAJÁ,NOVA FRIBURGO,BARUERI
Nome_do_subdistrito,GRANDE IBES,SÃO GONÇALO,SANTA MARIA,SANTA ROSA,SALTO,PACAJÁ,NOVA FRIBURGO,BARUERI
Nome_do_bairro,Nossa Senhora da Penha,Tribobó I,BRASÍLIA (todos os setores),SANTA ROSA (demais setores),SALTO (todos os setores),PACAJÁ (todos os setores),NOVA FRIBURGO (todos os setores),Boa Vista
populacao,514.0,1064.0,551.0,224.0,933.0,460.0,828.0,246.0
geometry,"POLYGON ((-40.3340742684591 -20.3545072660569, -40.3340595003196 -20.3545026423599, -40.331433539687694 -20.353680490971602, -40.325840211781 -20....","POLYGON ((-43.02064791596119 -22.8562301674079, -43.02155960175969 -22.8559012604845, -43.02164763902479 -22.855875203072998, -43.02259703090819 -...","POLYGON ((-48.02162766243939 -16.0264409342408, -48.0215355522151 -16.0265802804776, -48.0214387183895 -16.0267219885151, -48.0213477033925 -16.02...","POLYGON ((-54.5330173081123 -27.7617336606387, -54.526796398233 -27.761026573433597, -54.5200905584692 -27.7622160246069, -54.519174986498896 -27....","POLYGON ((-47.2749133123394 -23.2137086977407, -47.2749184880597 -23.2137083650352, -47.2749236226555 -23.2137077205546, -47.2749287006534 -23.213...","POLYGON ((-51.2310047570277 -3.38511150189605, -51.230256839695194 -3.38483566407569, -51.227856253939194 -3.38473122965021, -51.2238903434834 -3....","POLYGON ((-42.5368967507287 -22.3027755579253, -42.537058000768695 -22.302692900341796, -42.537266677291086 -22.302539780555897, -42.5374083126479...","POLYGON ((-46.871291 -23.493866, -46.871268 -23.493879, -46.870576 -23.494321, -46.870222 -23.494650999999998, -46.869968 -23.494854999999998, -46..."


In [88]:
br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310120 entries, 0 to 310119
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   Cod_setor            310120 non-null  object  
 1   domicilios           310120 non-null  int64   
 2   renda                310120 non-null  float64 
 3   Nome_da_UF           310120 non-null  object  
 4   Nome_do_municipio    310120 non-null  object  
 5   Nome_do_distrito     310120 non-null  object  
 6   Nome_do_subdistrito  310120 non-null  object  
 7   Nome_do_bairro       310120 non-null  object  
 8   populacao            310120 non-null  float64 
 9   geometry             310120 non-null  geometry
 10  area_ha              310120 non-null  float64 
dtypes: float64(3), geometry(1), int64(1), object(6)
memory usage: 26.0+ MB


Criação da densidade demográfica.


In [89]:
br['densidade'] = br['populacao'] / br['area_ha']

Criação da renda por domicílios


In [90]:
br['renda_dom'] = br['renda'] / br['domicilios']

In [91]:
br.sample(8).T

,89930,196167,297419,225398,291025,189815,76374,218311
Cod_setor,312490605000008,330590125000002,354780908000075,432200415000002,353900405000013,330455705290145,211220905000068,431490205001257
domicilios,137,89,268,325,80,422,245,351
renda,219901.0,97834.0,776991.0,501523.0,125422.0,15563224.0,277059.0,693310.0
Nome_da_UF,Minas Gerais,Rio de Janeiro,São Paulo,Rio Grande do Sul,São Paulo,Rio de Janeiro,Maranhão,Rio Grande do Sul
Nome_do_municipio,EUGENÓPOLIS,TRAJANO DE MORAES,SANTO ANDRÉ,TRIUNFO,PIRANGI,RIO DE JANEIRO,TIMON,PORTO ALEGRE
Nome_do_distrito,EUGENÓPOLIS,VISCONDE DE IMBÉ,CAPUAVA,PASSO RASO,PIRANGI,RIO DE JANEIRO,TIMON,PORTO ALEGRE
Nome_do_subdistrito,EUGENÓPOLIS,VISCONDE DE IMBÉ,CAPUAVA,PASSO RASO,PIRANGI,BARRA DA TIJUCA,TIMON,PORTO ALEGRE
Nome_do_bairro,EUGENÓPOLIS (todos os setores),TRAJANO DE MORAES (todos os setores),Jardim Ana Maria,TRIUNFO (todos os setores),PIRANGI (todos os setores),Barra da Tijuca,São Francisco II,Belém Novo
populacao,395.0,325.0,835.0,984.0,255.0,1271.0,964.0,1053.0
geometry,"POLYGON ((-42.178778581539405 -21.0974585093804, -42.1759318445783 -21.1000634118905, -42.1766110375516 -21.100729789901898, -42.177038151106 -21....","POLYGON ((-42.2060100363054 -22.100526548336695, -42.20669807947779 -22.0999871559807, -42.208426702525486 -22.099751595710295, -42.21042475300309...","POLYGON ((-46.48744 -23.624868, -46.486669 -23.624788, -46.486683 -23.624941999999997, -46.486664 -23.625082, -46.486554999999996 -23.625213, -46....","POLYGON ((-51.4940594048751 -29.8196263871826, -51.4922350570659 -29.8231895154486, -51.490250736134094 -29.8241220705678, -51.4887099353457 -29.8...","POLYGON ((-48.655878450397104 -21.093694889320798, -48.6533614843571 -21.0957916401759, -48.65076734990169 -21.098282199776296, -48.6478281640618 ...","POLYGON ((-43.3450479011546 -23.0110742442429, -43.34517792854889 -23.011072755310096, -43.349470979766195 -23.011023595938795, -43.34950931127149...","POLYGON ((-42.84520992 -5.087016, -42.84457704 -5.08693104, -42.84401904 -5.08685598, -42.84309204 -5.08670901, -42.84276084 -5.08707099, -42.8424...","POLYGON ((-51.162385816708 -30.1928112603022, -51.1625044411786 -30.1937485955294, -51.1625057167853 -30.1945237806602, -51.1628227256604 -30.1953..."


In [92]:
br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310120 entries, 0 to 310119
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   Cod_setor            310120 non-null  object  
 1   domicilios           310120 non-null  int64   
 2   renda                310120 non-null  float64 
 3   Nome_da_UF           310120 non-null  object  
 4   Nome_do_municipio    310120 non-null  object  
 5   Nome_do_distrito     310120 non-null  object  
 6   Nome_do_subdistrito  310120 non-null  object  
 7   Nome_do_bairro       310120 non-null  object  
 8   populacao            310120 non-null  float64 
 9   geometry             310120 non-null  geometry
 10  area_ha              310120 non-null  float64 
 11  densidade            310120 non-null  float64 
 12  renda_dom            310120 non-null  float64 
dtypes: float64(5), geometry(1), int64(1), object(6)
memory usage: 30.8+ MB


In [93]:
br_geo = br.loc[:,['Cod_setor', 'geometry']]

In [94]:
br_data = br.drop(columns=['geometry']).copy()

In [95]:
br_data.sample(10).T

,53844,251315,103022,132529,200110,227870,159067,19446,1880,229654
Cod_setor,320130810000180,355030846000357,315660105000004,251080805000137,240840905000004,420240405000159,410830405000318,291400005000072,270320505000032,420455805000019
domicilios,337,75,289,108,228,245,338,178,50,152
renda,471649.0,128481.0,285427.0,208765.0,176563.0,560052.0,445291.0,257940.0,43150.0,167358.0
Nome_da_UF,Espírito Santo,São Paulo,Minas Gerais,Paraiba,Rio Grande do Norte,Santa Catarina,Paraná,Bahia,Alagoas,Santa Catarina
Nome_do_municipio,CARIACICA,SÃO PAULO,RUBIM,PATOS,OLHO-D'ÁGUA DO BORGES,BLUMENAU,FOZ DO IGUAÇU,IPIRÁ,IGREJA NOVA,CORREIA PINTO
Nome_do_distrito,ITAQUARI,JARDIM SÃO LUÍS,RUBIM,PATOS,OLHO-D'ÁGUA DO BORGES,BLUMENAU,FOZ DO IGUAÇU,IPIRÁ,IGREJA NOVA,CORREIA PINTO
Nome_do_subdistrito,ITAQUARI,JARDIM SÃO LUÍS,RUBIM,PATOS,OLHO-D'ÁGUA DO BORGES,BLUMENAU,FOZ DO IGUAÇU,IPIRÁ,IGREJA NOVA,CORREIA PINTO
Nome_do_bairro,Operário,SÃO PAULO (todos os setores),RUBIM (todos os setores),PATOS (demais setores),OLHO-D'ÁGUA DO BORGES (todos os setores),ÁGUA VERDE,FOZ DO IGUAÇU (todos os setores),IPIRÁ (todos os setores),IGREJA NOVA (todos os setores),CORREIA PINTO (todos os setores)
populacao,1027.0,238.0,837.0,418.0,818.0,717.0,1169.0,567.0,178.0,327.0
area_ha,12.187422,1.323604,17.336357,179.574528,10654.166698,18.364721,50.700501,6.613488,18.933836,10697.224359


In [96]:
br_geo_gdf = gpd.GeoDataFrame(br_geo, geometry='geometry')

In [100]:
br_geo_gdf.sample(10)

,Cod_setor,geometry
179545,330350005170043,"POLYGON ((-43.60566 -22.83585, -43.60535 -22.83333, -43.60577 -22.83334, -43.60571 -22.83282, -43.60420 -22.83271, -43.60414 -22.83139, -43.60353 ..."
114858,510285005000018,"POLYGON ((-58.81907 -11.13737, -58.78132 -11.10738, -58.77564 -11.10902, -58.78013 -11.11340, -58.76561 -11.12913, -58.68760 -11.05540, -58.68399 ..."
179677,330350005180083,"POLYGON ((-43.51884 -22.69973, -43.51895 -22.69961, -43.51920 -22.69940, -43.51956 -22.69910, -43.51983 -22.69886, -43.51993 -22.69876, -43.51997 ..."
109136,317080010000005,"POLYGON ((-44.78524 -17.38252, -44.77747 -17.38219, -44.77305 -17.38179, -44.76929 -17.38286, -44.76634 -17.38380, -44.75387 -17.38327, -44.73791 ..."
206046,430222025000003,"POLYGON ((-53.78131 -28.48738, -53.78131 -28.48821, -53.77959 -28.48830, -53.77858 -28.48857, -53.77730 -28.49026, -53.77683 -28.49118, -53.77673 ..."
285140,352940105000322,"POLYGON ((-46.44351 -23.64773, -46.44335 -23.64783, -46.44323 -23.64790, -46.44306 -23.64800, -46.44263 -23.64829, -46.44246 -23.64840, -46.44222 ..."
47376,231320335000022,"POLYGON ((-40.39995 -4.85917, -40.39905 -4.86036, -40.39850 -4.86252, -40.39974 -4.86517, -40.40025 -4.87074, -40.40025 -4.87349, -40.39759 -4.874..."
238750,280030805000080,"POLYGON ((-37.04624 -10.98964, -37.04626 -10.98965, -37.04773 -10.99166, -37.04896 -10.99285, -37.04991 -10.99388, -37.04994 -10.99390, -37.05134 ..."
92648,313190105000023,"POLYGON ((-43.81207 -20.24851, -43.81182 -20.24885, -43.81157 -20.24917, -43.81137 -20.24944, -43.81115 -20.24982, -43.81126 -20.25000, -43.81148 ..."
90282,312620805000024,"POLYGON ((-46.32645 -15.11608, -46.32106 -15.11775, -46.31935 -15.11776, -46.31874 -15.11848, -46.31665 -15.11873, -46.31494 -15.11970, -46.31384 ..."


In [97]:
type(br_geo_gdf)

geopandas.geodataframe.GeoDataFrame

In [98]:
br_geo_gdf.to_parquet('./artifacts/br_geo_gdf.parquet')

In [99]:
br_data.to_csv('./artifacts/br_data.csv', index=False)